In [6]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from src.call_finder import *


# Community Peer Explorer Demo

This notebook is an interactive demo of the Locus Peer Explorer package. 
It allows you to find and explore the peers of any MSA or county in 2016. 

## Identifying geographic peers

The first type of peer we can identify are geographic peers.
This feature returns the communities which are the closest to you geographically.

In [7]:
geo = interact_manual(show_peers, df_county_dist=fixed(df_county_dist), df_msa_def= fixed(df_msa_def),area=input_area, n_peers=input_n_peers, year=fixed(2016))


interactive(children=(Dropdown(description='Area of interest', index=250, layout=Layout(width='80%'), options=…

## Identifying peers with user selected industries and outcomes

This second feature allows the user to find its peers based on any subset of industries and outcomes. This will return the communities that have the most similar concentration of those industries and closest level of outcomes.

To select or unselect multiple industries or outcomes, hold the cmd key. 

The graphs allow to compare:
* the industry concentration of the user's community to its peers, to the national mean and 1st and 3rd quantile (in orange).
* the distribution of industry concentration within the peer group and among all other communities.

In [8]:
input_peers = interact_manual(show_fms_peers, area=input_area, year=fixed(2016), n_peers=input_n_peers,fms=input_fms, outcomes=input_outcomes)


interactive(children=(Dropdown(description='Area of interest', index=250, layout=Layout(width='80%'), options=…

## Identifying peers with algorithmically selected industries

For all the next appoaches, we automatically select a subset of industries and find the communities that are the most similar to you on this subset of features using a distance metric. 

#### Most concentrated industries 

This feature will return the peers based on the industries that are the most prevalent in your community. You will be able to specify the number of industries you want to base the peer identification on.
  

In [9]:
top_fms = interact_manual(show_top_fms_peers,area=input_area, year=fixed(2016), n_peers=input_n_peers, n_fms=input_n_fms, filter_pop = input_population, save_fig='./results/')


interactive(children=(Dropdown(description='Area of interest', index=250, layout=Layout(width='80%'), options=…

#### Most distinguishing industries

The peers returned here are based on the industries that are distinguishing for your community. It will include the most present industries but also the least present. Industries are ranked based on how much their concentration in your community differs from other communities. You can choose how many of those distinguishing features you want to base your analysis on.

In [10]:
disting = interact_manual(show_disting_peers, area=input_area,  year=fixed(2016), n_peers=input_n_peers, n_feat=input_n_features, filter_pop = input_population, save_fig='./results/')


interactive(children=(Dropdown(description='Area of interest', index=250, layout=Layout(width='80%'), options=…